# NOTES/UPDATES

1. MAKE SURE YOUR FILE IS CUBIC SYMMMETRY
2. RUN ANY NEEDED CODE
3. THIS PROGRAM IS IN BETA, BUGS AND ERRORS MAY OCCUR, EMAIL stephenlimzy2008@gmail.com TO REPORT BUGS

### Import all libraries needed (RUN THIS SCRIPT FIRST)

In [13]:
import numpy as np #a classic library used for math functions
import ase #used for visualization
from ase import Atoms
from ase.visualize import view #to view the structure
from ase.io import read, write #to read .CIF or .XYZ files
from ase import io #in case

### Define File and retrieve A-Site atom and B-Site atom (NEEDED)

In [14]:
FileName = "LaFeO3" #replace with your name
FileType = ".cif" #default is .cif replace with your file type
FilePath = FileName+FileType
print(FilePath)
File = read(FilePath) #be sure to define this as your file
CompoundA = ''
CompoundB = ''
Move = 0
for letter in FileName:
    if letter.isupper():
          Move += 1
    if Move == 1:
            CompoundA = CompoundA + letter
    if Move == 2:
          CompoundB = CompoundB + letter
print(CompoundA, CompoundB)
    



#AlternativeFile = read("YourFile.xyz")



LaFeO3.cif
La Fe


### Convert .cif to .xyz (NEEDED)

Depending on your needs, you may want to render your .CIF file, so to do that in VMD, we first convert to .XYZ

In [15]:
write(FileName + ".xyz", File)
File = FileName + ".xyz"

### View the file (Recommended)

In [16]:
Test = read(File)

view(Test)

<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>

### Supercell the original file and rotate accordingly(NEEDED)

In [17]:
#making huge sample to delete from
# from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view

# Read the original structure from an XYZ file
atom = read(File)

rotation_offset = 0 #auto 0 at first
rotate_scale = 90 #we going by increments of 90


supercell_atoms = make_supercell(atom, [[3, 0, 0], [0, 3, 0], [0, 0, 3]])
corner_atoms = []
while CompoundA not in corner_atoms:
    corner_atoms = []
#while the corner atoms list doesnt contain the A-site atom do:
    supercell_atoms.rotate(rotation_offset,'z')
    print(rotation_offset)
    # Extract minimum x, y, and max z coordinate
    min_x = np.max(supercell_atoms.positions[:,0])
    min_y = np.max(supercell_atoms.positions[:,1])
    max_z = np.min(supercell_atoms.positions[:,2])
   
    #not the cleanest code, however below, we are just saying if it is -0.00000000 then just go ahead and mark it as 0
    # , if you dont, you will get a number that does not work, like -4*10^-16
    #the whole point of the code below is to retrieve the element that is at bottom-left most after rotating
    #x,y coordinates of smallest possible atom
    for atom in supercell_atoms:
        if atom.position[0] <= min_x:
            if f"{atom.position[0]:.8f}" == "-0.00000000" :
                min_x = -0.00000000
            else:
                min_x = atom.position[0]
        if atom.position[1] <= min_y:
            if f"{atom.position[1]:.8f}" == "-0.00000000" :
                min_y = -0.00000000
            else:
                min_y = atom.position[1]
    
    #locate corner atom name            
    for atom in supercell_atoms:
        if f"{atom.position[0]:.8f}" == f"{min_x:.8f}" and f"{atom.position[1]:.8f}"== f"{min_y:.8f}":
            corner_atoms.append(atom.symbol)
            print(corner_atoms)
    
    # move the molecule into the vacuum
    # move the molecule into the center of the vacuum
    if rotation_offset>0:
        displacement = [-min_x, -min_y, 0]
        supercell_atoms.translate(displacement)
    # Move the molecule into the vacuum
    rotation_offset = rotation_offset+rotate_scale
    #catom is corner atom
#we search for the A-site atom



write('SS_' + FileName + '.xyz', supercell_atoms)

# Visualize the supercell
#view(supercell_atoms)
view(supercell_atoms)

0
['La']
['La', 'La']
['La', 'La', 'La']


<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>

### Make base by removing atoms along x = 0 and y=0 (NEEDED)

In [18]:

# Read XYZ file into ASE Atoms object
Bush = read('SS_' + FileName + '.xyz')
lattice_info = None
General = 'General_' + FileName + '.xyz'
with open(General, 'w') as f:
    f.write('')
    # Print each line from the XYZ file
with open('SS_' + FileName + '.xyz', 'r') as f:
    for line in f:
        if 'Lattice=' in line:
            lattice_info = line.strip()

AllElements = [atom.symbol for atom in Bush]
TypesOfElements = []
for Elements in AllElements:
    if Elements not in TypesOfElements:
        TypesOfElements.append(Elements)
ElementCorrespondentList = {}
for Element in TypesOfElements:
    with open('SS_' + FileName + '.xyz', 'r') as f:
        for line in f:
            if Element in line:
                Number = line[len(line)-2] #extra space at the end
                ElementCorrespondentList[Element] = Number
print(ElementCorrespondentList)

def atom_to_xyz_line(atom):
    symbol = atom.symbol
    coordinates = atom.position
    if len(symbol) == 2:
        line = f"{symbol}      {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    else:
        line = f"{symbol}       {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    return line
# unoticable but important spacing for the file

atomNumbers = 0

for atom in Bush:
    x = atom.position[0]
    y = atom.position[1]
    z = atom.position[2]
    if x != 0 and y!=0:
        atomNumbers = atomNumbers + 1

with open(General, 'a') as f:
    f.write(f"{atomNumbers}\n")

with open(General, 'a') as f:
    f.write(f"{lattice_info}\n")


for atom in Bush:
    x = atom.position[0]
    y = atom.position[1]
    z = atom.position[2]
    if x != 0 and y!=0:
        
        with open(General, 'a') as f:
            # Example usage:
            xyz_line = atom_to_xyz_line(atom)
            f.write(xyz_line)
        #write('General_' + FileName + '.xyz', atom)


#start vacuum
# Read the molecular structure from 'General' file
GeneralFile = read(General)

# Get atomic positions to determine molecule extents
positions = GeneralFile.get_positions()

# Find the minimum and maximum coordinates of all atoms
min_coords = np.min(GeneralFile.positions, axis=0)
max_coords = np.max(GeneralFile.positions, axis=0)

# Calculate the dimensions of the molecule
mol_size_x = max_coords[0] - min_coords[0]
mol_size_y = max_coords[1] - min_coords[1]
mol_size_z = max_coords[2] - min_coords[2]

# Calculate the dimensions of the vacuum box to fully enclose the rotated molecule
vacuum_size_x = max_coords[0] - min_coords[0]
vacuum_size_y = max_coords[1] - min_coords[1]
vacuum_size_z = max_coords[2] - min_coords[2]

# Shift the atoms to position the center of the vacuum box at one of the corners of the molecule
center_x_rotated = min_coords[0]
center_y_rotated = min_coords[1]
center_z_rotated = min_coords[2]
GeneralFile.positions -= [center_x_rotated, center_y_rotated, center_z_rotated]

# Set the cell parameters of the vacuum box
GeneralFile.set_cell([vacuum_size_x, vacuum_size_y, vacuum_size_z])

# Write the modified structure back to 'General' file
write(General, GeneralFile)

#we vacuum the structure tightly

view(read(General))

{'La': '0', 'Fe': '1', 'O': '2'}


<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>

### Declare layers to make Site Base (NEEDED)

This is seperate code because it is a very complicated process

In [19]:
# identify the layers using id's of elements
TypeOfZ = []
for atom in GeneralFile:
    ULI = atom.position[2] #unique layer identifier
    if ULI not in TypeOfZ:
        TypeOfZ.append(ULI)
# we are essentially getting all layers coordinates, then we can determine the atoms info in that layer by assigning each layer
# a specific list/matrix
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    globals()[ZLayerName] = []
    for atom in GeneralFile:
        if atom.position[2] == Z:
            if [atom.symbol,atom.position[0],atom.position[1],atom.index] not in globals()[ZLayerName]:
                globals()[ZLayerName].append([atom.symbol,atom.position[0],atom.position[1],atom.index])
    print(f"{ZLayerName}:{globals()[ZLayerName]}")

GreatestXVal = 0
GreatestYVal = 0
IndexesToDelete = []
#find greatest X and Y
for atom in GeneralFile:
    if atom.position[0] > GreatestXVal:
        GreatestXVal = atom.position[0]
    if atom.position[1] > GreatestXVal:
        GreatestYVal = atom.position[1]

#loop through looking for indexes with those greatest x or y
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    for atom_info in globals()[ZLayerName]:
        if GreatestXVal in atom_info and atom_info[3] not in IndexesToDelete:
            IndexesToDelete.append(atom_info[3])
print(IndexesToDelete)
#add atoms info into specific z layer
#0th index is symbol, x is 1st index, y is 2nd index, index OF ATOM is 3rd index
atomNumbersG = 0
for atom in GeneralFile:
    atomNumbersG = atomNumbersG + 1
SiteBase = "SB_" + FileName + ".xyz"
with open(SiteBase, 'w') as f:
    f.write('')
atomNumbersG = atomNumbersG - len(IndexesToDelete)

def atom_to_xyz_line(atom):
    symbol = atom.symbol
    coordinates = atom.position
    if len(symbol) == 2:
        line = f"{symbol}      {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    else:
        line = f"{symbol}       {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    return line


#prep titles
with open('General_' + FileName + '.xyz', 'r') as f:
    for line in f:
        if 'Lattice=' in line:
            lattice_info = line.strip()
with open(SiteBase, 'a') as f:
    f.write(f"{atomNumbersG}\n")

with open(SiteBase, 'a') as f:
    f.write(f"{lattice_info}\n")

for atom in GeneralFile:
    if atom.index not in IndexesToDelete:
        with open(SiteBase, 'a') as f:
            f.write(f"{atom_to_xyz_line(atom)}")
            
ViewBase = read(SiteBase)
view(ViewBase)

Z_1.9785885:[['Fe', 0.0, 0.0, 0], ['Fe', 0.0, 3.9571769999999997, 6], ['O', 0.0, 1.9785885, 7], ['Fe', 0.0, 7.914354, 15], ['O', 0.0, 5.9357655000000005, 16], ['Fe', 3.9571769999999997, 0.0, 24], ['O', 1.9785885, 0.0, 25], ['Fe', 3.9571769999999997, 3.9571769999999997, 34], ['O', 1.9785885, 3.9571769999999997, 35], ['O', 3.9571769999999997, 1.9785885, 36], ['Fe', 3.9571769999999997, 7.914354, 49], ['O', 1.9785885, 7.914354, 50], ['O', 3.9571769999999997, 5.9357655000000005, 51], ['Fe', 7.914354, 0.0, 63], ['O', 5.9357655000000005, 0.0, 64], ['Fe', 7.914354, 3.9571769999999997, 73], ['O', 5.9357655000000005, 3.9571769999999997, 74], ['O', 7.914354, 1.9785885, 75], ['Fe', 7.914354, 7.914354, 88], ['O', 5.9357655000000005, 7.914354, 89], ['O', 7.914354, 5.9357655000000005, 90]]
Z_0.0:[['O', 0.0, 0.0, 1], ['O', 0.0, 3.9571769999999997, 8], ['O', 0.0, 7.914354, 17], ['O', 3.9571769999999997, 0.0, 26], ['La', 1.9785885, 1.9785885, 33], ['O', 3.9571769999999997, 3.9571769999999997, 37], ['La'

<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>

### Make A-Site(NEEDED)

In [20]:
#all layers
AllLayers = []
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    AllLayers.append(ZLayerName)
AllLayers = sorted(AllLayers)
print(AllLayers)

#second we read the z layers
BSiteLayers = [] #aka possible Layers to delete for a site
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    for atominfo in globals()[ZLayerName]:
        if atominfo[0] == CompoundB and ZLayerName not in BSiteLayers: #we get the B-site layers
            BSiteLayers.append(ZLayerName)
print("B-Layers:" , BSiteLayers)


ASiteLayers = [] #aka possible Layers to delete for b site
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    for atominfo in globals()[ZLayerName]:
        if atominfo[0] == CompoundA and ZLayerName not in ASiteLayers: #we get the A-site layers
            ASiteLayers.append(ZLayerName)
print("A-Layers:" , ASiteLayers)
VisualizeLayers = []
for Layer in AllLayers:
    if Layer in BSiteLayers:
        VisualizeLayers.append("B")
    elif Layer in ASiteLayers:
        VisualizeLayers.append("A")


#lets make A site first
if VisualizeLayers[0] == 'B':
    AtomsLayer = AllLayers[0]+"_SB"
    print(AllLayers[0])
elif VisualizeLayers[len(VisualizeLayers)-1] == 'B':
    AtomsLayer = AllLayers[len(VisualizeLayers)-1]+"_SB"
    print(AllLayers[len(VisualizeLayers)-1])
print(VisualizeLayers)
for ZLayer in AllLayers:
    ZLayerNameSB = str(ZLayer)+"_SB"
    globals()[ZLayerNameSB] = []

for atom in read(SiteBase):
    ZLayerNameSB = "Z_"+str(atom.position[2])+"_SB"
    globals()[ZLayerNameSB].append([atom.symbol,atom.position[0],atom.position[1],atom.index])


FileA = "ASite_"+FileName + ".xyz"
FileB = "BSite_"+FileName + ".xyz"

AtomsOnLayer = globals()[AtomsLayer]


def atom_to_xyz_line(atom):
    symbol = atom.symbol
    coordinates = atom.position
    if len(symbol) == 2:
        line = f"{symbol}      {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    else:
        line = f"{symbol}       {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    return line
print(AtomsOnLayer)

with open(FileA, 'w') as f:
    f.write('')
#prep titles
with open('SB_' + FileName + '.xyz', 'r') as f:
    for line in f:
        if 'Lattice=' in line:
            lattice_info = line.strip()

atomNumbersL = len(read(SiteBase))-len(AtomsOnLayer)
with open(FileA, 'a') as f:
    f.write(f"{atomNumbersL}\n")

with open(FileA, 'a') as f:
    f.write(f"{lattice_info}\n")

for atom in read(SiteBase):
    if [atom.symbol, atom.position[0], atom.position[1], atom.index] not in AtomsOnLayer:
        with open(FileA, 'a') as f:
            f.write(atom_to_xyz_line(atom))
atoms = read(FileA)
cell_params = atoms.get_cell()
# Modify the cell dimensions to change vacuum size (adjust along the z-axis)
vacuum_size = 10.0  # Adjust this value to change the size of the vacuum
cell_params[2, 2] += vacuum_size  # Increase or decrease vacuum size along z-axis
# Calculate current middle along the z-axis
middle_z = (cell_params[2, 2] + cell_params[0, 2]) / 2.0

# Calculate shift needed to center the structure
shift_z = middle_z - np.mean(atoms.positions[:, 2])

# Shift positions of all atoms to center the vacuum
atoms.positions[:, 2] += shift_z

# Set the modified cell parameters back to the structure
atoms.set_cell(cell_params)
write(FileA, atoms)
view(read(FileA))

['Z_0.0', 'Z_1.9785885', 'Z_3.957177', 'Z_5.9357655', 'Z_7.914354', 'Z_9.8929425']
B-Layers: ['Z_1.9785885', 'Z_5.9357655', 'Z_9.8929425']
A-Layers: ['Z_0.0', 'Z_3.957177', 'Z_7.914354']
Z_9.8929425
['A', 'B', 'A', 'B', 'A', 'B']
[['Fe', 0.0, 0.0, 4], ['Fe', 0.0, 3.957177, 12], ['O', 0.0, 1.9785885, 13], ['O', 0.0, 5.9357655, 17], ['Fe', 3.957177, 0.0, 24], ['O', 1.9785885, 0.0, 25], ['Fe', 3.957177, 3.957177, 38], ['O', 1.9785885, 3.957177, 39], ['O', 3.957177, 1.9785885, 40], ['O', 3.957177, 5.9357655, 47], ['O', 5.9357655, 0.0, 50], ['O', 5.9357655, 3.957177, 56]]


<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>

### Make B-Site (NEEDED)

In [21]:
#all layers
AllLayers = []
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    AllLayers.append(ZLayerName)
AllLayers = sorted(AllLayers)
print(AllLayers)

#second we read the z layers
BSiteLayers = [] #aka possible Layers to delete for a site
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    for atominfo in globals()[ZLayerName]:
        if atominfo[0] == CompoundB and ZLayerName not in BSiteLayers: #we get the B-site layers
            BSiteLayers.append(ZLayerName)
print("B-Layers:" , BSiteLayers)


ASiteLayers = [] #aka possible Layers to delete for b site
for Z in TypeOfZ:
    ZLayerName = "Z_"+str(Z)
    for atominfo in globals()[ZLayerName]:
        if atominfo[0] == CompoundA and ZLayerName not in ASiteLayers: #we get the A-site layers
            ASiteLayers.append(ZLayerName)
print("A-Layers:" , ASiteLayers)
VisualizeLayers = []
for Layer in AllLayers:
    if Layer in BSiteLayers:
        VisualizeLayers.append("B")
    elif Layer in ASiteLayers:
        VisualizeLayers.append("A")


#lets make B site second
if VisualizeLayers[0] == 'A':
    AtomsLayer = AllLayers[0]+"_SB"
    print(AllLayers[0])
elif VisualizeLayers[len(VisualizeLayers)-1] == 'A':
    AtomsLayer = AllLayers[len(VisualizeLayers)-1]+"_SB"
    print(AllLayers[len(VisualizeLayers)-1])
print(VisualizeLayers)
for ZLayer in AllLayers:
    ZLayerNameSB = str(ZLayer)+"_SB"
    globals()[ZLayerNameSB] = []

for atom in read(SiteBase):
    ZLayerNameSB = "Z_"+str(atom.position[2])+"_SB"
    globals()[ZLayerNameSB].append([atom.symbol,atom.position[0],atom.position[1],atom.index])

FileB = "BSite_"+FileName + ".xyz"

AtomsOnLayer = globals()[AtomsLayer]


def atom_to_xyz_line(atom):
    symbol = atom.symbol
    coordinates = atom.position
    if len(symbol) == 2:
        line = f"{symbol}      {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    else:
        line = f"{symbol}       {coordinates[0]:.8f}         {coordinates[1]:.8f}        {coordinates[2]:.8f}         {ElementCorrespondentList[atom.symbol]}\n"
    return line
print(AtomsOnLayer)

with open(FileB, 'w') as f:
    f.write('')
#prep titles
with open('SB_' + FileName + '.xyz', 'r') as f:
    for line in f:
        if 'Lattice=' in line:
            lattice_info = line.strip()

atomNumbersL = len(read(SiteBase))-len(AtomsOnLayer)
with open(FileB, 'a') as f:
    f.write(f"{atomNumbersL}\n")

with open(FileB, 'a') as f:
    f.write(f"{lattice_info}\n")

for atom in read(SiteBase):
    if [atom.symbol, atom.position[0], atom.position[1], atom.index] not in AtomsOnLayer:
        with open(FileB, 'a') as f:
            f.write(atom_to_xyz_line(atom))
atoms = read(FileB)
cell_params = atoms.get_cell()
# Modify the cell dimensions to change vacuum size (adjust along the z-axis)
vacuum_size = 10.0  # Adjust this value to change the size of the vacuum
cell_params[2, 2] += vacuum_size  # Increase or decrease vacuum size along z-axis
# Calculate current middle along the z-axis
middle_z = (cell_params[2, 2] + cell_params[0, 2]) / 2.0

# Calculate shift needed to center the structure
shift_z = middle_z - np.mean(atoms.positions[:, 2])

# Shift positions of all atoms to center the vacuum
atoms.positions[:, 2] += shift_z

# Set the modified cell parameters back to the structure
atoms.set_cell(cell_params)
write(FileB, atoms)
view(read(FileB))

['Z_0.0', 'Z_1.9785885', 'Z_3.957177', 'Z_5.9357655', 'Z_7.914354', 'Z_9.8929425']
B-Layers: ['Z_1.9785885', 'Z_5.9357655', 'Z_9.8929425']
A-Layers: ['Z_0.0', 'Z_3.957177', 'Z_7.914354']
Z_0.0
['A', 'B', 'A', 'B', 'A', 'B']
[['O', 0.0, 0.0, 1], ['O', 0.0, 3.957177, 8], ['O', 3.957177, 0.0, 20], ['La', 1.9785885, 1.9785885, 27], ['O', 3.957177, 3.957177, 31], ['La', 1.9785885, 5.9357655, 42], ['La', 5.9357655, 1.9785885, 51], ['La', 5.9357655, 5.9357655, 57]]


<Popen: returncode: None args: ['c:\\Users\\coolp\\AppData\\Local\\Programs\...>